In [1]:
pip install keras_vggface

In [ ]:
from keras_vggface.vggface import VGGFace
from keras.engine import  Model
from keras.layers import Flatten, Dense, Input
from keras_vggface.vggface import VGGFace
nb_class=56
vgg_model = VGGFace(model='resnet50',weights='vggface',include_top=False, input_shape=(224, 224, 3))
last_layer = vgg_model.get_layer('avg_pool').output
x = Flatten(name='flatten')(last_layer)
out = Dense(nb_class, activation='softmax', name='classifier')(x)
custom_vgg_model = Model(vgg_model.input, out)


In [ ]:
for layer in custom_vgg_model.layers[:150]:
 layer.trainable = False

for layer in custom_vgg_model.layers:
 print(layer, layer.trainable)

<keras.engine.input_layer.InputLayer object at 0x7f4b131b0b38> False
<keras.layers.convolutional.Conv2D object at 0x7f4b131b0ba8> False
<keras.layers.normalization.BatchNormalization object at 0x7f4b131b0e10> False
<keras.layers.core.Activation object at 0x7f4b131b0b70> False
<keras.layers.pooling.MaxPooling2D object at 0x7f4b131b0f60> False
<keras.layers.convolutional.Conv2D object at 0x7f4b131b0f28> False
<keras.layers.normalization.BatchNormalization object at 0x7f4b134614e0> False
<keras.layers.core.Activation object at 0x7f4b13243470> False
<keras.layers.convolutional.Conv2D object at 0x7f4b1317b860> False
<keras.layers.normalization.BatchNormalization object at 0x7f4b136c79e8> False
<keras.layers.core.Activation object at 0x7f4b136c7400> False
<keras.layers.convolutional.Conv2D object at 0x7f4b137b7b70> False
<keras.layers.convolutional.Conv2D object at 0x7f4b1379fe48> False
<keras.layers.normalization.BatchNormalization object at 0x7f4b137a7d68> False
<keras.layers.normalization

In [ ]:
custom_vgg_model.summary()

Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1/7x7_s2 (Conv2D)           (None, 112, 112, 64) 9408        input_6[0][0]                    
__________________________________________________________________________________________________
conv1/7x7_s2/bn (BatchNormaliza (None, 112, 112, 64) 256         conv1/7x7_s2[0][0]               
__________________________________________________________________________________________________
activation_246 (Activation)     (None, 112, 112, 64) 0           conv1/7x7_s2/bn[0][0]            
____________________________________________________________________________________________

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
train_datagen1 = ImageDataGenerator(rescale=1./255, validation_split=0.02)

validation_datagen = ImageDataGenerator(rescale=1./255)
image_size=224
train_dir='/content/drive/My Drive/dataset1'
train_batchsize = 128

val_batchsize = 8

train_generator = train_datagen.flow_from_directory(
train_dir,
target_size=(image_size, image_size),
batch_size=train_batchsize,
class_mode='categorical')
validation_generator = train_datagen.flow_from_directory(
    train_dir, # same directory as training data
    target_size=(image_size, image_size),
    batch_size=val_batchsize,
    class_mode='categorical',
    subset='validation') # set as validation data
validation_generator1 = train_datagen1.flow_from_directory(
    train_dir, # same directory as training data
    target_size=(image_size, image_size),
    batch_size=val_batchsize,
    class_mode='categorical',
    subset='validation') # set as validation data


Found 6409 images belonging to 56 classes.
Found 1268 images belonging to 56 classes.
Found 113 images belonging to 56 classes.


In [ ]:
from keras import optimizers
custom_vgg_model.compile(loss='categorical_crossentropy',optimizer=optimizers.RMSprop(lr=1e-4),metrics=['acc'])
history = custom_vgg_model.fit(validation_generator,
steps_per_epoch=
train_generator.samples/train_generator.batch_size,epochs=1,verbose=1)


Epoch 1/1
51/50 [==============================] - 16s 311ms/step - loss: 2.0875 - acc: 0.5490


In [ ]:
a=custom_vgg_model.predict(validation_generator1)


ValueError: ignored

In [ ]:
from sklearn.metrics import accuracy_score
print(accuracy_score(validation_generator1.classes,a))

ValueError: ignored

In [ ]:
from keras.models import model_from_json

model_json = custom_vgg_model.to_json()
with open("/content/drive/My Drive/model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
custom_vgg_model.save_weights("/content/drive/My Drive/model.h5")
print("Saved model to disk")